In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
import os
print(os.listdir("C:\\Users\\user\\Documents\\jupyter..python\\deeplearning practice\\Attention\\sentiment_labelled_sentences"))


['.DS_Store', 'amazon_cells_labelled.txt', 'imdb_labelled.txt', 'readme.txt', 'yelp_labelled.txt']


In [3]:
data1=open("C:\\Users\\user\\Documents\\jupyter..python\\deeplearning practice\\Attention\\sentiment_labelled_sentences\\amazon_cells_labelled.txt")

In [4]:
text=data1.read()

In [5]:
import re
sent=re.findall(r'.*\n',text)

In [6]:
sent=[(line[:-3],int(line[-2])) for line in sent]

In [7]:
df=pd.DataFrame(sent,columns=['review','score'])

In [8]:
t=tf.keras.preprocessing.text.Tokenizer()
t.fit_on_texts(df.review)
text_matrix=t.texts_to_sequences(df.review)

In [9]:
len_mat=[]
for i in range(len(text_matrix)):
    len_mat.append(len(text_matrix[i]))

In [10]:
from keras.preprocessing.sequence import pad_sequences
text_pad=pad_sequences(text_matrix,maxlen=32,padding="post")

In [23]:
            text_pad[5]

array([  2,  22,   8, 816,   1, 155,   8,  70,   4,   8, 273,  55, 145,
         8,  70, 339, 146,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0])

In [11]:
inputs1=tf.keras.layers.Input(shape=(32,))
x1=tf.keras.layers.Embedding(1879,32,
             input_length=32,
            embeddings_regularizer=tf.keras.regularizers.l2(.001))(inputs1)
x1=tf.keras.layers.LSTM(100,dropout=0.3,recurrent_dropout=0.2)(x1)
outputs1=tf.keras.layers.Dense(1,activation='sigmoid')(x1)
Model=tf.keras.Sequential()
model1=tf.keras.Model(inputs1,outputs1)

In [12]:
model1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 32, 32)            60128     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 113,429
Trainable params: 113,429
Non-trainable params: 0
_________________________________________________________________


In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df.review,df.score,test_size=0.3,random_state=3)

In [14]:
model1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy())
model1.fit(text_pad,df.score,batch_size=100,epochs=10,verbose=1,shuffle=True,validation_split=0.2)

Epoch 1/10


C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


8/8 [==============================] - 3s 336ms/step - loss: 0.7354 - val_loss: 0.7267
Epoch 2/10
8/8 [==============================] - 1s 146ms/step - loss: 0.7171 - val_loss: 0.7132
Epoch 3/10
8/8 [==============================] - 1s 111ms/step - loss: 0.7058 - val_loss: 0.7051
Epoch 4/10
8/8 [==============================] - 1s 108ms/step - loss: 0.6994 - val_loss: 0.6994
Epoch 5/10
8/8 [==============================] - 1s 103ms/step - loss: 0.6958 - val_loss: 0.6968
Epoch 6/10
8/8 [==============================] - 1s 103ms/step - loss: 0.6940 - val_loss: 0.6976
Epoch 7/10
8/8 [==============================] - 1s 106ms/step - loss: 0.6932 - val_loss: 0.6957
Epoch 8/10
8/8 [==============================] - 1s 110ms/step - loss: 0.6819 - val_loss: 0.6541
Epoch 9/10
8/8 [==============================] - 1s 106ms/step - loss: 0.5349 - val_loss: 0.4873
Epoch 10/10
8/8 [==============================] - 1s 104ms/step - loss: 0.3274 - val_loss: 0.4733


In [15]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as k
class attention(Layer):
    def __init__(self):
        super(attention,self).__init__()
    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")
        super(attention,self).build(input_shape)
    def call(self,x):
        e=k.squeeze(k.tanh(k.dot(x,self.W)+self.b),axis=-1)
        a=k.softmax(e)
        a=k.expand_dims(a,axis=-1)
        output=x*a
        return k.sum(output,axis=1)
    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()
    

In [16]:
inputs1=tf.keras.layers.Input(shape=(32,))
x1=tf.keras.layers.Embedding(1879,32,
             input_length=32,
            embeddings_regularizer=tf.keras.regularizers.l2(.001))(inputs1)
x1=tf.keras.layers.LSTM(100,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(x1)
att_out=attention()(x1)
outputs1=tf.keras.layers.Dense(1,activation='sigmoid',trainable=True)(att_out)
Model=tf.keras.Sequential()
model1=tf.keras.Model(inputs1,outputs1)

In [17]:
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 32, 32)            60128     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 100)           53200     
_________________________________________________________________
attention (attention)        (None, 100)               132       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 113,561
Trainable params: 113,561
Non-trainable params: 0
_________________________________________________________________


In [18]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df.review,df.score,test_size=0.3,random_state=3)

In [20]:
model1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy())
model1.fit(text_pad,df.score,batch_size=100,epochs=50,verbose=1,shuffle=True,validation_split=0.2)

Epoch 1/50
8/8 [==============================] - 2s 236ms/step - loss: 0.1052 - val_loss: 0.5400
Epoch 2/50
8/8 [==============================] - 1s 134ms/step - loss: 0.0832 - val_loss: 0.6676
Epoch 3/50
8/8 [==============================] - 1s 107ms/step - loss: 0.0516 - val_loss: 0.7465
Epoch 4/50
8/8 [==============================] - 1s 105ms/step - loss: 0.0568 - val_loss: 0.6751
Epoch 5/50
8/8 [==============================] - 1s 107ms/step - loss: 0.0419 - val_loss: 0.5985
Epoch 6/50
8/8 [==============================] - 1s 108ms/step - loss: 0.0436 - val_loss: 0.6031
Epoch 7/50
8/8 [==============================] - 1s 107ms/step - loss: 0.0382 - val_loss: 0.8823
Epoch 8/50
8/8 [==============================] - 1s 110ms/step - loss: 0.0301 - val_loss: 0.8901
Epoch 9/50
8/8 [==============================] - 1s 111ms/step - loss: 0.0235 - val_loss: 0.6745
Epoch 10/50
8/8 [==============================] - 1s 110ms/step - loss: 0.0249 - val_loss: 0.7891
Epoch 11/50
8/8 [==